# 로컬에서 훈련 하기
- https://www.kaggle.com/code/mitanshuchakrawarty/fine-tune-llm-for-text-summary

## 1. 환경 셋업

In [1]:
!huggingface-cli login --token ""

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/ec2-user/.cache/huggingface/token
Login successful


In [2]:
import os 
os.environ['TRANSFORMERS_CACHE'] = "/home/ec2-user/SageMaker/.cache" 
os.environ['HF_DATASETS_CACHE'] = "/home/ec2-user/SageMaker/.cache" 
os.environ['HF_HOME'] = "/home/ec2-user/SageMaker/.cache"

In [3]:
%store -r data_folder
%store -r train_data_json 
%store -r validation_data_json 
%store -r test_data_json 

print("data_folder: ", data_folder)
print("train_data_json: ", train_data_json)
print("validation_data_json: ", validation_data_json)
print("test_data_json: ", test_data_json)

data_folder:  ../data/cnn_dailymail
train_data_json:  ../data/cnn_dailymail/train_dataset.json
validation_data_json:  ../data/cnn_dailymail/validation_dataset.json
test_data_json:  ../data/cnn_dailymail/test_dataset.json


In [4]:
import torch
import time
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

from datasets import Dataset, load_dataset
from datasets import load_dataset, load_metric
from transformers import pipeline, set_seed
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import warnings
warnings.filterwarnings("ignore")

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


## 2. 베이스 모델 준비

In [19]:
model_id = "meta-llama/Meta-Llama-3-8B"
output_dir = "/home/ec2-user/SageMaker/models/llama-3-8b-cnn"

### Config YAML 파일 생성

In [20]:
%%writefile llama_3_8b_fsdp_qlora.yaml
# script parameters
model_id:  "meta-llama/Meta-Llama-3-8B" # Hugging Face model id
dataset_path: "../data/cnn_dailymail"                      # path to dataset
max_seq_len:  2048              # max sequence length for model and packing of the dataset
# training parameters
output_dir: "/home/ec2-user/SageMaker/models/llama-3-8b-cnn" # Temporary output directory for model checkpoints
report_to: "tensorboard"               # report metrics to tensorboard
learning_rate: 0.0002                  # learning rate 2e-4
lr_scheduler_type: "constant"          # learning rate scheduler
num_train_epochs: 1                    # number of training epochs
per_device_train_batch_size: 1         # batch size per device during training
per_device_eval_batch_size: 1          # batch size for evaluation
gradient_accumulation_steps: 2         # number of steps before performing a backward/update pass
optim: adamw_torch                     # use torch adamw optimizer
logging_steps: 10                      # log every 10 steps
save_strategy: epoch                   # save checkpoint every epoch
evaluation_strategy: epoch             # evaluate every epoch
max_grad_norm: 0.3                     # max gradient norm
warmup_ratio: 0.03                     # warmup ratio
bf16: true                             # use bfloat16 precision
tf32: true                             # use tf32 precision
gradient_checkpointing: true           # use gradient checkpointing to save memory
# FSDP parameters: https://huggingface.co/docs/transformers/main/en/fsdp
fsdp: "full_shard auto_wrap offload" # remove offload if enough GPU memory
fsdp_config:
  backward_prefetch: "backward_pre"
  forward_prefetch: "false"
  use_orig_params: "false"

Overwriting llama_3_8b_fsdp_qlora.yaml


## 3. 훈련 Script 실행

In [21]:
!ACCELERATE_USE_FSDP=1 FSDP_CPU_RAM_EFFICIENT_LOADING=1 torchrun --nproc_per_node=4 \
../scripts/run_fsdp_qlora.py \
--config llama_3_8b_fsdp_qlora.yaml

[2024-06-27 10:53:11,066] torch.distributed.run: [WARNING] 
[2024-06-27 10:53:11,066] torch.distributed.run: [WARNING] *****************************************
[2024-06-27 10:53:11,066] torch.distributed.run: [WARNING] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
[2024-06-27 10:53:11,066] torch.distributed.run: [WARNING] *****************************************
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  

## 4. 베이스 모델과 훈련된 모델 머지

In [22]:
model_id, output_dir

('meta-llama/Meta-Llama-3-8B',
 '/home/ec2-user/SageMaker/models/llama-3-8b-cnn')

### 모델 머지 및 로컬에 저장

In [23]:
from peft import AutoPeftModelForCausalLM
import torch

# Load PEFT model on CPU

model = AutoPeftModelForCausalLM.from_pretrained(
    output_dir,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)  
# Merge LoRA and base model and save
merged_model = model.merge_and_unload()
merged_model.save_pretrained(output_dir,safe_serialization=True, max_shard_size="2GB")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### 머지된 모델 로딩

In [24]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer 


# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  pretrained_model_name_or_path = output_dir,
  torch_dtype=torch.float16,
  quantization_config= {"load_in_4bit": True},
  device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(output_dir)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## 5. 추론

### 테스트 데이터 셋 로딩

In [25]:
from datasets import load_dataset 
from random import randint

# Load our test dataset
eval_dataset = load_dataset("json", data_files=test_data_json, split="train")
rand_idx = randint(0, len(eval_dataset))
messages = eval_dataset[rand_idx]["messages"][:2]
rand_idx, messages

(1,
 [{'content': 'You are an AI assistant specialized in news articles.Your role is to provide accurate summaries and insights.Please analyze the given text and provide concise, informative summaries that highlight the key goals and findings.',
   'role': 'system'},
  {'content': "Please summarize the goals for journalist in this text:\n\nEuropes largest budget airline has just been granted approval to develop a transatlantic service between Europe and America. Ryanair, the Ireland-based budget carrier, plans to run services between key European cities and the US with one-way tickets costing from as little as $15. The idea is part of an aggressive growth strategy, connecting airports such as London Stansted and Berlin with New York, Boston and Miami. Ryanair, Europe's largest budget airline, has received board approval to develop a transatlantic service for budget passengers, with one-way tickets between the US and key European cities costing as little as $15 . The airline first moote

### 추론

In [27]:
# Test on sample 
input_ids = tokenizer.apply_chat_template(messages,add_generation_prompt=True,return_tensors="pt").to(model.device)
outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id= tokenizer.eos_token_id,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
response = outputs[0][input_ids.shape[-1]:]

print(f"**Query:**\n{eval_dataset[rand_idx]['messages'][1]['content']}\n")
print(f"**Original Answer:**\n{eval_dataset[rand_idx]['messages'][2]['content']}\n")
print(f"**Generated Answer:**\n{tokenizer.decode(response,skip_special_tokens=True)}")

# **Query:**
# How long was the Revolutionary War?
# **Original Answer:**
# The American Revolutionary War lasted just over seven years. The war started on April 19, 1775, and ended on September 3, 1783. 
# **Generated Answer:**
# The Revolutionary War, also known as the American Revolution, was an 18th-century war fought between the Kingdom of Great Britain and the Thirteen Colonies. The war lasted from 1775 to 1783.

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


**Query:**
Please summarize the goals for journalist in this text:

Europes largest budget airline has just been granted approval to develop a transatlantic service between Europe and America. Ryanair, the Ireland-based budget carrier, plans to run services between key European cities and the US with one-way tickets costing from as little as $15. The idea is part of an aggressive growth strategy, connecting airports such as London Stansted and Berlin with New York, Boston and Miami. Ryanair, Europe's largest budget airline, has received board approval to develop a transatlantic service for budget passengers, with one-way tickets between the US and key European cities costing as little as $15 . The airline first mooted the idea of offering transatlantic flights back in 2008 and board approval comes after the airline has invested 18 months in cleaning up its image and improving customer service. It is believed it could take up to five years to realize the plan, as the airline needs to so